## 기업 데이터에 추가 데이터 붙이기

In [1]:
import pandas as pd

giup = pd.read_csv('../data/giup_data.csv')

In [2]:
giup = giup.iloc[:,1:]

In [3]:
giup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 65 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   회사명              533 non-null    object 
 1   stock_code       533 non-null    int64  
 2   업종               532 non-null    object 
 3   상장주선인            533 non-null    object 
 4   공모가 (원)          533 non-null    float64
 5   공모금액 (천원)        533 non-null    object 
 6   최초상장주식수 (주)      533 non-null    object 
 7   call             533 non-null    float64
 8   kospi            533 non-null    object 
 9   modified_esi     533 non-null    float64
 10  6개월 확약           533 non-null    float64
 11  3개월 확약           533 non-null    float64
 12  1개월 확약           533 non-null    float64
 13  15일 확약           533 non-null    float64
 14  미확약              533 non-null    float64
 15  수요예측_최종값         533 non-null    float64
 16  회사명_bs           533 non-null    object 
 17  corp_code       

In [ ]:
# 변환, 원자료
kospi = pd.read_csv('../notion_data/코스피지수.csv')

In [ ]:
# 변환, 원자료 
kosdaq = pd.read_csv('../notion_data/코스닥지수.csv')

In [ ]:
# 변환, 원자료 
money = pd.read_csv('../notion_data/주요국통화.csv')

In [14]:
import pandas as pd
import numpy as np


def compute_pre_listing_returns_calendar(giup, kospi, kosdaq, money, window_days=20):
    """
    For each company's listing date, collect daily returns of KOSPI, KOSDAQ, and USD/KRW
    for all trading days falling within the calendar window [listing_date - window_days, listing_date).

    Parameters:
    - giup: DataFrame with ['회사명', '상장일']
    - kospi, kosdaq, money: DataFrames with ['변환', '원자료']
    - window_days: number of calendar days to look back (default=20)

    Returns:
    - DataFrame in long format:
      ['회사명', '상장일', 'index', '변환', 'return']
      where 'index' is one of 'kospi', 'kosdaq', 'money'.
    """
    # Prepare company dates
    giup = giup.copy()
    giup['상장일'] = pd.to_datetime(giup['상장일'])

    # Prepare market data: set 변환 index, convert to numeric, and compute returns
    def prepare(df):
        df = df.copy()
        # Ensure correct types
        df['변환'] = pd.to_datetime(df['변환'])
        df['원자료'] = pd.to_numeric(df['원자료'], errors='coerce')
        # Sort, index by date
        df = df.sort_values('변환').set_index('변환')
        # Compute daily pct change, skip NaNs
        df['return'] = df['원자료'].pct_change()
        return df['return']

    series_map = {
        'kospi': prepare(kospi),
        'kosdaq': prepare(kosdaq),
        'money': prepare(money)
    }

    records = []
    # Iterate each company
    for _, row in giup.iterrows():
        event_date = row['상장일']
        start_date = event_date - pd.Timedelta(days=window_days)

        for name, series in series_map.items():
            # Select window and drop NaN returns
            window = series.loc[start_date:event_date - pd.Timedelta(days=1)].dropna()
            for dt, ret in window.iteritems():
                records.append({
                    '회사명': row['회사명'],
                    '상장일': event_date,
                    'index': name,
                    '변환': dt,
                    'return': ret
                })

    # Return assembled DataFrame
    return pd.DataFrame(records)

# Example usage:
# df_long = compute_pre_listing_returns_calendar(giup, kospi, kosdaq, money, window_days=20)
# print(df_long.head())


In [15]:
compute_pre_listing_returns_calendar(giup, kospi, kosdaq, money, window_days=20)

C:\Users\Owner\AppData\Local\Temp\ipykernel_42160\241844599.py:33: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['return'] = df['원자료'].pct_change()
C:\Users\Owner\AppData\Local\Temp\ipykernel_42160\241844599.py:33: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['return'] = df['원자료'].pct_change()
C:\Users\Owner\AppData\Local\Temp\ipykernel_42160\241844599.py:33: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill N

AttributeError: 'Series' object has no attribute 'iteritems'

In [ ]:
# 
na_return = pd.read_csv('../notion_data/Nasdaq.csv')

In [49]:
giup.rename(columns={'Daily Return (%)' : '나스닥_수익률'},inplace=True)

In [52]:
giup.rename(columns={'원자료' : '코스피_지수'},inplace=True)

In [58]:
giup.rename(columns={'원자료' : '코스닥_지수'},inplace=True)